In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import tqdm
import math
import tensorboard
from models import RNN, LSTM  # models.py에서 RNN, LSTM 모델 가져오기
from dataset import StockDataset  # dataset.py에서 KOSPI 데이터 가져오기
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter

In [7]:
from torch.utils.tensorboard import SummaryWriter

In [3]:
!pip uninstall tensorflow-tensorboard

In [4]:
!pip install tensorboard

In [3]:
!pip uninstall -y tensorboard tb-nightly

Found existing installation: tensorboard 2.15.0
Uninstalling tensorboard-2.15.0:


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\torch\lib\shutil.py", line 816, in move
    os.rename(src, real_dst)
PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'c:\\users\\user\\anaconda3\\envs\\torch\\scripts\\tensorboard.exe' -> 'C:\\Users\\Public\\Documents\\ESTsoft\\CreatorTemp\\pip-uninstall-m7_t41vu\\tensorboard.exe'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\torch\lib\site-packages\pip\_internal\cli\base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
  File "C:\Users\user\anaconda3\envs\torch\lib\site-packages\pip\_internal\commands\uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
  File "C:\Users\user\anaconda3\envs\torch\lib\site-packages\pip\_internal\req\req_install.py", line 687, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "C

In [4]:
!pip install tb-nightly

  Obtaining dependency information for tb-nightly from https://files.pythonhosted.org/packages/f5/a8/0586ef9ed6d13b55ae41dd693b7b49caa01c3e020e60f87694b33292017d/tb_nightly-2.16.0a20231026-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB 487.6 kB/s eta 0:00:12
   --- ------------------------------------ 0.5/5.5 MB 4.7 MB/s eta 0:00:02
   ------- -------------------------------- 1.0/5.5 MB 8.3 MB/s eta 0:00:01
   ------- -------------------------------- 1.0/5.5 MB 8.3 MB/s eta 0:00:01
   ------- -------------------------------- 1.0/5.5 MB 8.3 MB/s eta 0:00:01
   ------------- -------------------------- 1.9/5.5 MB 6.4 MB/s eta 0:00:01
   -------------------------------- ------- 4.5/5.5 MB 13.1 MB/s eta 0:00:01
   --------------------------------- ------ 4.7/5.5 MB 12.9 MB/s eta 0:00:01
   --------------------------------

ERROR: Could not install packages due to an OSError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\Users\\user\\anaconda3\\envs\\torch\\Scripts\\tensorboard.exe' -> 'C:\\Users\\user\\anaconda3\\envs\\torch\\Scripts\\tensorboard.exe.deleteme'
Consider using the `--user` option or check the permissions.



In [5]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\torch\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\user\anaconda3\envs\torch\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\user\anaconda3\envs\torch\Scripts\tensorboard.exe\__main__.py", line 7, in <module>
  File "C:\Users\user\anaconda3\envs\torch\lib\site-packages\tensorboard\main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "C:\Users\user\anaconda3\envs\torch\lib\site-packages\absl\app.py", line 308, in run
    _run_main(main, args)
  File "C:\Users\user\anaconda3\envs\torch\lib\site-packages\absl\app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "C:\Users\user\anaconda3\envs\torch\lib\site-packages\tensorboard\program.py", line 276, in main
    return runner(self.flags) or 

In [47]:
stock_file = 'KOSPI.csv'

In [9]:
def l2_loss(pred, label):
    loss = torch.nn.functional.mse_loss(pred, label, reduction='mean')  # MSE Loss 사용
    return loss

In [10]:
def train(model, dataloader, optimizer):
    # 모델을 학습 단계로 설정
    model.train()
    loader = tqdm.tqdm(dataloader)  

    loss_epoch = 0
    
    for idx, (data, label) in enumerate(loader):    
        data, label = data.float(), label.float()
        output = model(data)  # batch_size(64) 별로 전체 데이터를 나누어서 학습 진행
        optimizer.zero_grad()  # epoch 한 번의 학습이 완료되어지면 gradient를 항상 0으로 초기화
        loss = l2_loss(output, label)  # 손싫 함수는 MSE Loss로 설정
        loss.backward()  # 역전파 진행
        optimizer.step()  # 역전파 단계에서 수집된 변화도로 매개변수 조정
        loss_epoch += loss.detach().item()  # 각 epoch 별 loss 출력
    loss_epoch /= len(loader)
    return loss_epoch

In [11]:
def eval(model, dataloader):
    # 모델을 평가 단계로 설정
    model.eval()
    loader = tqdm.tqdm(dataloader)

    loss_epoch = 0

    for idx, (data, label) in enumerate(loader):
        data, label = data.float(), label.float()
        output = model(data)  # batch_size(64) 별로 전체 데이터를 나누어서 평가 진행
        loss = l2_loss(output, label)  # 손실 함수는 MSE Loss로 설정
        loss_epoch += loss.detach().item()  # 각 epoch 별 loss 출력
    loss_epoch /= len(loader)
    return loss_epoch

In [54]:
def main():
    # 데이터 셋 설정
    dataset_train = StockDataset(file_path = stock_file, time_step = 10)
    dataset_test = StockDataset(file_path = stock_file, time_step = 10, train_flag=False)

    train_loader = DataLoader(dataset_train, batch_size=64, shuffle=True)
    test_loader = DataLoader(dataset_test, batch_size=64, shuffle=False)
    
    # RNN, LSTM 중 원하는 모델 설정
    model = RNN(rnn_layer=2, input_size=1, hidden_size=4)
    model = LSTM(lstm_layer=2, input_dim=1, hidden_size=8)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # optimizer는 Adam으로 설정
    total_epoch = 101  # 100 epoch까지 checkpoints 및 결과 plot 생성

    for epoch_idx in range(total_epoch):
        train_loss = train(model, train_loader, optimizer)  # 학습을 진행해 train_loss 계산
        print("stage: train, epoch:{:5d}, loss:{}".format(epoch_idx, train_loss))  # 각 step마다 loss 출력
        logger.add_scalar('Train/Loss', train_loss, epoch_idx)  # tensorboardX의 add_scaler를 활용해 각 step에서의 학습 손실 값을 스칼라 형태로 기록

        if epoch_idx % 10 == 0:  # 10 에폭마다 평가용 데이터로 검증
            eval_loss = eval(model, test_loader)  # 평가를 진행해 eval_loss 계산
            print("stage: test, epoch:{:5d}, loss:{}".format(epoch_idx, eval_loss))  # 10번의 step마다 loss 출력
            # torch.save(model.state_dict(), "{}/_KOSPI_LSTM/checkpoint_{:0>3}.ckpt".format(checkpointdir, epoch_idx))  # 모델의 학습 가중치를 checkpoints로 저장
            
            torch.save(model.state_dict(), "KOSPI_LSTM/checkpoint_{:0>3}.ckpt".format(epoch_idx))  # 모델의 학습 가중치를 checkpoints로 저장
            logger.add_scalar('Test/Loss', eval_loss, epoch_idx)  # tensorboardX의 add_scaler를 활용해 각 step에서의 평가 손실 값을 스칼라 형태로 기록


In [49]:
import logging

In [50]:
import logger

In [51]:
from torch.utils.tensorboard import SummaryWriter

logger = SummaryWriter()

In [52]:
checkpointdir = './checkpoints'
logger = SummaryWriter(checkpointdir)

In [55]:
if __name__ == '__main__':
    main()

data len:9921
data len:1102


100%|████████████████████████████████████████████████████████████████████████████████| 155/155 [00:01<00:00, 77.52it/s]


stage: train, epoch:    0, loss:0.3299548755970693


100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 242.92it/s]


stage: test, epoch:    0, loss:0.8320198617875576


...

stage: train, epoch:   99, loss:0.00037590574835532253


100%|███████████████████████████████████████████████████████████████████████████████| 155/155 [00:01<00:00, 150.41it/s]


stage: train, epoch:  100, loss:0.0003681528294691816


100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 426.22it/s]

stage: test, epoch:  100, loss:0.03441367646802165


In [46]:
import os
os.getcwd()

'C:\\Users\\user\\Desktop\\주식'